In [41]:
import torch
from safetensors.torch import load_file

In [42]:
ct_movq_dict = dict(torch.load("/home/mlfavorfit/Desktop/lib_link/favorfit/kjg/0_model_weights/diffusion/Kandinsky/models--kandinsky-community--kandinsky-2-2-controlnet-depth/snapshots/4ecd717e8c9086cf4a16ca28b64894f70a42cd08/movq/diffusion_pytorch_model.bin"))
ct_unet_dict = dict(torch.load("/home/mlfavorfit/Desktop/lib_link/favorfit/kjg/0_model_weights/diffusion/Kandinsky/models--kandinsky-community--kandinsky-2-2-controlnet-depth/snapshots/4ecd717e8c9086cf4a16ca28b64894f70a42cd08/unet/diffusion_pytorch_model.bin"))

un_movq_dict = dict(load_file("/home/mlfavorfit/Desktop/lib_link/favorfit/kjg/0_model_weights/diffusion/Kandinsky/models--kandinsky-community--kandinsky-2-2-decoder-inpaint/snapshots/db790ad5cbcabed886f069ef2710774657621702/movq/diffusion_pytorch_model.safetensors"))
un_unet_dict = dict(load_file("/home/mlfavorfit/Desktop/lib_link/favorfit/kjg/0_model_weights/diffusion/Kandinsky/models--kandinsky-community--kandinsky-2-2-decoder-inpaint/snapshots/db790ad5cbcabed886f069ef2710774657621702/unet/diffusion_pytorch_model.safetensors"))

In [43]:
ct_movq = sorted([(cur, ct_movq_dict[cur].shape)for cur in ct_movq_dict], key=lambda x:x[0])
ct_unet = sorted([(cur, ct_unet_dict[cur].shape)for cur in ct_unet_dict], key=lambda x:x[0])
un_movq = sorted([(cur, un_movq_dict[cur].shape)for cur in un_movq_dict], key=lambda x:x[0])
un_unet = sorted([(cur, un_unet_dict[cur].shape)for cur in un_unet_dict], key=lambda x:x[0])

In [44]:
for ct, un in zip(ct_movq, un_movq):
    if ct != un: print("warn")

In [45]:
only_ct = []
common_ct_un = []
for name, shape in ct_unet:
    if name in un_unet_dict.keys():
        common_ct_un.append(name)
    else:
        only_ct.append(name)

In [46]:
for name in common_ct_un:
    if un_unet_dict[name].shape != ct_unet_dict[name].shape: print(name)

conv_in.weight


In [47]:
un_unet_dict["conv_in.weight"].shape

torch.Size([384, 9, 3, 3])

In [48]:
ct_unet_dict["conv_in.weight"].shape

torch.Size([384, 8, 3, 3])

In [49]:
un_unet_dict["conv_in.weight"].shape

torch.Size([384, 9, 3, 3])

In [53]:
for cur in only_ct:
    print(cur, ct_unet_dict[cur].shape)

add_embedding.input_hint_block.0.bias torch.Size([16])
add_embedding.input_hint_block.0.weight torch.Size([16, 3, 3, 3])
add_embedding.input_hint_block.10.bias torch.Size([96])
add_embedding.input_hint_block.10.weight torch.Size([96, 96, 3, 3])
add_embedding.input_hint_block.12.bias torch.Size([256])
add_embedding.input_hint_block.12.weight torch.Size([256, 96, 3, 3])
add_embedding.input_hint_block.14.bias torch.Size([4])
add_embedding.input_hint_block.14.weight torch.Size([4, 256, 3, 3])
add_embedding.input_hint_block.2.bias torch.Size([16])
add_embedding.input_hint_block.2.weight torch.Size([16, 16, 3, 3])
add_embedding.input_hint_block.4.bias torch.Size([32])
add_embedding.input_hint_block.4.weight torch.Size([32, 16, 3, 3])
add_embedding.input_hint_block.6.bias torch.Size([32])
add_embedding.input_hint_block.6.weight torch.Size([32, 32, 3, 3])
add_embedding.input_hint_block.8.bias torch.Size([96])
add_embedding.input_hint_block.8.weight torch.Size([96, 32, 3, 3])


In [82]:
import torch

# 기존 텐서의 모양과 새로운 모양 정의
old_shape = (384, 9, 3, 3)
new_shape = (384, 13, 3, 3)

# 새로운 텐서 생성 및 기존 텐서 값 복사
new_weight = torch.zeros(new_shape)  # 새로운 텐서를 모두 0으로 초기화
old_weight = un_unet_dict["conv_in.weight"]
new_weight[:, :old_shape[1], :, :] = old_weight

# 추가된 부분에 랜덤한 값 채우기
random_part = torch.rand((new_shape[0], new_shape[1] - old_shape[1], new_shape[2], new_shape[3]))
new_weight[:, old_shape[1]:, :, :] = random_part

# 결과 확인
print(new_weight.shape)


torch.Size([384, 13, 3, 3])


In [83]:
un_unet_dict["conv_in.weight"] = new_weight

In [84]:
for cur in only_ct:
    un_unet_dict[cur] = ct_unet_dict[cur]

In [85]:
print(len(un_unet_dict))
print(un_unet_dict["conv_in.weight"].shape)

740
torch.Size([384, 13, 3, 3])


In [68]:
from safetensors.torch import save_file

save_file(un_unet_dict, "/home/mlfavorfit/Desktop/lib_link/favorfit/kjg/0_model_weights/diffusion/Kandinsky/kandinsky-inpainting-controlnet/unet/diffusion_pytorch_model.safetensors")

In [70]:
temp_dict = load_file("/home/mlfavorfit/Desktop/lib_link/favorfit/kjg/0_model_weights/diffusion/Kandinsky/kandinsky-inpainting-controlnet/unet/diffusion_pytorch_model.safetensors")

In [71]:
print(len(temp_dict))
print(temp_dict["conv_in.weight"].shape)

740
torch.Size([384, 13, 3, 3])


In [87]:
for cur in only_ct:
    print(cur, temp_dict[cur].shape)

add_embedding.input_hint_block.0.bias torch.Size([16])
add_embedding.input_hint_block.0.weight torch.Size([16, 3, 3, 3])
add_embedding.input_hint_block.10.bias torch.Size([96])
add_embedding.input_hint_block.10.weight torch.Size([96, 96, 3, 3])
add_embedding.input_hint_block.12.bias torch.Size([256])
add_embedding.input_hint_block.12.weight torch.Size([256, 96, 3, 3])
add_embedding.input_hint_block.14.bias torch.Size([4])
add_embedding.input_hint_block.14.weight torch.Size([4, 256, 3, 3])
add_embedding.input_hint_block.2.bias torch.Size([16])
add_embedding.input_hint_block.2.weight torch.Size([16, 16, 3, 3])
add_embedding.input_hint_block.4.bias torch.Size([32])
add_embedding.input_hint_block.4.weight torch.Size([32, 16, 3, 3])
add_embedding.input_hint_block.6.bias torch.Size([32])
add_embedding.input_hint_block.6.weight torch.Size([32, 32, 3, 3])
add_embedding.input_hint_block.8.bias torch.Size([96])
add_embedding.input_hint_block.8.weight torch.Size([96, 32, 3, 3])


In [1]:
from transformers import CLIPImageProcessor, CLIPVisionModelWithProjection
image_processor = CLIPImageProcessor.from_pretrained("/home/mlfavorfit/Desktop/lib_link/favorfit/kjg/0_model_weights/diffusion/Kandinsky/kandinsky-inpainting-controlnet"
        , subfolder="image_processor"
    )

In [2]:
from PIL import Image
image_processor(Image.open("/home/mlfavorfit/Desktop/style6.jpg")).pixel_values[0]


array([[[1.4631865, 1.4631865, 1.4631865, ..., 1.9157377, 1.9157377,
         1.9157377],
        [1.4631865, 1.4631865, 1.4485881, ..., 1.9157377, 1.9157377,
         1.9157377],
        [1.4631865, 1.4631865, 1.4485881, ..., 1.9157377, 1.9157377,
         1.9157377],
        ...,
        [1.5069818, 1.5069818, 1.5069818, ..., 1.8865409, 1.9011393,
         1.9011393],
        [1.5069818, 1.5069818, 1.5069818, ..., 1.9011393, 1.9157377,
         1.9157377],
        [1.5069818, 1.5069818, 1.5069818, ..., 1.9011393, 1.9157377,
         1.9157377]],

       [[1.1293944, 1.1293944, 1.1293944, ..., 1.4595653, 1.4595653,
         1.4595653],
        [1.1293944, 1.1293944, 1.1143867, ..., 1.4595653, 1.4595653,
         1.4595653],
        [1.1293944, 1.1293944, 1.1143867, ..., 1.4595653, 1.4595653,
         1.4595653],
        ...,
        [1.1293944, 1.1293944, 1.1293944, ..., 1.219441 , 1.2344488,
         1.2344488],
        [1.1293944, 1.1293944, 1.1293944, ..., 1.2344488, 1.2494565,
   

In [1]:
from transformers import CLIPImageProcessor, CLIPVisionModelWithProjection
from diffusers import DDPMScheduler, UNet2DConditionModel, VQModel, KandinskyV22PriorPipeline
import torch

movq = VQModel.from_pretrained(
            "/home/mlfavorfit/Desktop/lib_link/favorfit/kjg/0_model_weights/diffusion/Kandinsky/kandinsky-inpainting-controlnet", subfolder="movq", torch_dtype=torch.float16
        ).eval()
image_encoder = CLIPVisionModelWithProjection.from_pretrained(
    "/home/mlfavorfit/Desktop/lib_link/favorfit/kjg/0_model_weights/diffusion/Kandinsky/kandinsky-inpainting-controlnet", subfolder="image_encoder", torch_dtype=torch.float16
).eval()

/home/mlfavorfit/anaconda3/envs/diffusion_rnd/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
unet = UNet2DConditionModel.from_pretrained("/home/mlfavorfit/Desktop/lib_link/favorfit/kjg/0_model_weights/diffusion/Kandinsky/kandinsky-inpainting-controlnet", subfolder="unet")

In [5]:
torch.cuda.empty_cache()

In [37]:
movq.requires_grad_(False)
movq.to("cuda")
image_encoder.requires_grad_(False)
image_encoder.to("cuda")
print()

In [38]:
unet.requires_grad_(True)
unet.to("cuda")
print()

In [39]:
total_params = sum(p.numel() for p in unet.parameters())
total_params += sum(p.numel() for p in movq.parameters())
total_params += sum(p.numel() for p in image_encoder.parameters())

In [40]:
total_params

3166186251

In [31]:
from glob import glob
fns = glob("/media/mlfavorfit/sda/product_mask/*/*")

In [32]:
from PIL import Image
sorted_fns = sorted(fns)

In [33]:
imgs = []
masks = []
for idx, cur in enumerate(sorted_fns):
    if idx % 2 == 0:
        imgs.append(cur)
    else:
        masks.append(cur)

In [4]:
root_dir = "/media/mlfavorfit/sda/product_512"
import os
from tqdm import tqdm
from utils import make_outpaint_condition

imgs = sorted(glob("/media/mlfavorfit/sda/product_512/image/*"))
masks = sorted(glob("/media/mlfavorfit/sda/product_512/mask/*"))

for img, mask in tqdm(zip(imgs, masks), total=len(imgs)):
    img_pil = Image.open(img).convert("RGB")
    mask_pil = Image.open(mask).convert("RGB")

    save_img_path = os.path.join(root_dir,"object",os.path.basename(img))
    
    object_img = make_outpaint_condition(img_pil, mask_pil)
    object_img.save(save_img_path)

  0%|          | 636/197967 [02:33<11:49:25,  4.64it/s]/home/mlfavorfit/anaconda3/envs/diffusion_rnd/lib/python3.10/site-packages/PIL/Image.py:3186: DecompressionBombWarning: Image size (144953856 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
  0%|          | 892/197967 [03:35<13:33:50,  4.04it/s]/home/mlfavorfit/anaconda3/envs/diffusion_rnd/lib/python3.10/site-packages/PIL/Image.py:3186: DecompressionBombWarning: Image size (91850210 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
  1%|          | 1162/197967 [04:41<11:41:12,  4.68it/s]/home/mlfavorfit/anaconda3/envs/diffusion_rnd/lib/python3.10/site-packages/PIL/Image.py:3186: DecompressionBombWarning: Image size (100000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
  1%|          | 2085/197967 [08:29<14:36:00,  3.73it/s]/home/mlfavorfit/anaconda3/envs/diffusion_rnd/lib/python